In [249]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin

# Top Rated Movie 크롤링

In [291]:
base_url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
res = req.get("https://www.imdb.com/chart/top/?ref_=nv_mv_250")
html_text = bs(res.text,"html.parser")
num=250


In [292]:
title_list=html_text.find_all("td",{"class":"titleColumn"})

title_list =[x.text.strip().replace("\n",'').split('      ')[1].split('(')[0] for x in title_list]

title_list = title_list[:num]

In [293]:
rank_list = np.arange(num) + 1

In [294]:
year_list = html_text.find_all("span",{"class":"secondaryInfo"})
year_list = [x.text for x in year_list]

year_list = year_list[:num]

In [295]:
rating_list = html_text.find_all("strong")
rating_list = [x.text for x in rating_list]

rating_list=rating_list[:num]

# 세부 내용 크롤링 

In [296]:
href_list = html_text.find_all("td",{"class":"titleColumn"})
href_list = [x.find("a").get("href") for x in href_list]

url_list = [urljoin(base_url,x) for x in href_list]
url_list= url_list[:num]

In [297]:
genres_list = []
runtime_list =[]
director_list= []
total_revenue_list = []
for x in url_list:
    # x=url_list[0]
    res = req.get(x)
    html_text = bs(res.text,"html.parser")
    # 장르 리스트
    genres = html_text.find_all("div",{"class":"see-more inline canwrap"})
    genres=[x.find("a") for x in genres]
    genres = genres[1].text.strip()
    genres_list.append(genres)
    # 상영시간 리스트
    runtime = html_text.find_all("time")
    runtime_list.append(runtime[0].text.strip())
    #감독 리스트
    director = html_text.find("div",{"class":"credit_summary_item"}).find("a").text
    director_list.append(director)
    #총 수입
    total_revenue = html_text.find_all("div",{"id":"titleDetails"})
    total_revenue = total_revenue[0].find_all("div",{"class":"txt-block"})
    total_revenue = [x.text.strip().replace("\n","").split(":") for x in total_revenue]
    temp="None"
    for i in range(len(total_revenue)):
        if(total_revenue[i][0]=="Cumulative Worldwide Gross"):
            temp=total_revenue[i][1].strip()
            break
    total_revenue_list.append(temp)
# total_revenue[9].text.split(' ')[3]
# total_revenue_list.append(total_revenue)


In [298]:
# df_list=[];
# df_list.append(title_list)
# df_list.append(year_list)
# df_list.append(rating_list)

df = pd.DataFrame(title_list,index=rank_list,columns=["title"])
df["year"] = year_list
df["rating"] = rating_list
df["genres"] = genres_list
df["runtime"] = runtime_list
df["director"] = director_list
df["total revenue"] = total_revenue_list


df.index.names=["rank"]
df.columns.names=["Info"]

df

Info,title,year,rating,genres,runtime,director,total revenue
rank,,,,,,,
1,The Shawshank Redemption,(1994),9.2,Drama,2h 22min,Frank Darabont,"$28,815,245"
2,Daeboo,(1972),9.1,Crime,2h 55min,Francis Ford Coppola,"$246,120,974"
3,The Godfather: Part II,(1974),9.0,Crime,3h 22min,Francis Ford Coppola,"$48,035,783"
4,The Dark Knight,(2008),9.0,Action,2h 32min,Christopher Nolan,"$1,005,455,211"
5,12 Angry Men,(1957),8.9,Crime,1h 36min,Sidney Lumet,$576
...,...,...,...,...,...,...,...
246,The Terminator,(1984),8.0,Action,1h 47min,James Cameron,"$78,680,331"
247,Trois couleurs: Rouge,(1994),8.0,Drama,1h 39min,Krzysztof Kieslowski,"$3,581,969"
248,Moksoriui hyeongtae,(2016),8.0,Animation,2h 10min,Naoko Yamada,"$30,490,447"


In [299]:
df.to_csv("text.csv",sep=",",mode="w",encoding="utf-8-sig")